# Cleaning and geocoding pharmacy data

This notebook cleans data scraped from the New York State Department of Education’s Office of the Professions [online verification search engine](http://www.op.nysed.gov/opsearches.htm#rx). It also geocodes the data to generate latitude and longitude coordinates from street addresses using the [HERE API](https://developer.here.com/).

In [86]:
import pandas as pd
import requests
import wget
import re
import numpy as np
from getpass import getpass

pd.set_option('display.max_columns', None)

In [ ]:
%load_ext jupyternotify

In [88]:
pharmacies = pd.read_csv('pharmacy-data-scrape-6-22.csv', na_values = ['Not on file'],parse_dates=['date_first_registered', 
                                                                                                   'registration_begins',
                                                                                                   'registration_ends'])

### Dropping pharmacies with suppressed info

In [94]:
pharmacies.drop(pharmacies[(pharmacies.legal_name.str.contains('XX')) & ((pharmacies.trade_name.str.contains('CVS')==False)\
                          | (pharmacies.trade_name.isna()))].index, inplace=True)

### Cleaning dates, dropping non-retail pharmacies and adding an "active" flag next to open pharmacies

In [95]:
print('earliest date registered: ' + str(pharmacies.date_first_registered.min()))
print('latest date registered: ' + str(pharmacies.date_first_registered.max()))
print('earliest registration expiration: ' + str(pharmacies.registration_ends.min()))
print('latest registration expiration: ' + str(pharmacies.registration_ends.max()))

earliest date registered: 1972-01-01 00:00:00
latest date registered: 2071-12-29 00:00:00
earliest registration expiration: 1983-10-18 00:00:00
latest registration expiration: 2025-10-31 00:00:00


In [96]:
# Num with registration dates in the future
pharmacies.query('date_first_registered > "2022-06-22"').shape

(1287, 13)

In [97]:
# Changing century on dates in the future
pharmacies.loc[pharmacies.date_first_registered > "2022-06-22", 'date_first_registered'] = \
pharmacies.date_first_registered - pd.DateOffset(years=100)

In [98]:
pharmacies.query('date_first_registered > registration_ends').shape

(4, 13)

In [99]:
pharmacies.query('date_first_registered > registration_begins').shape

(11, 13)

In [100]:
print('earliest date registered: ' + str(pharmacies.date_first_registered.min()))
print('latest date registered: ' + str(pharmacies.date_first_registered.max()))
print('earliest registration begins: ' + str(pharmacies.date_first_registered.min()))
print('latest registration begins: ' + str(pharmacies.date_first_registered.max()))
print('earliest registration expiration: ' + str(pharmacies.registration_ends.min()))
print('latest registration expiration: ' + str(pharmacies.registration_ends.max()))

earliest date registered: 1927-12-05 00:00:00
latest date registered: 2022-06-16 00:00:00
earliest registration begins: 1927-12-05 00:00:00
latest registration begins: 2022-06-16 00:00:00
earliest registration expiration: 1983-10-18 00:00:00
latest registration expiration: 2025-10-31 00:00:00


In [101]:
pharmacies.establishment_status.value_counts()

DISCONTINUED    5493
TRANSFER        3613
ACTIVE             3
Name: establishment_status, dtype: int64

In [102]:
# No odd registration dates on currently active pharmacies
pharmacies.query('(registration_begins > registration_ends) & establishment_status.isna()')

,type,legal_name,trade_name,street_address,registration_number,date_first_registered,registration_begins,registration_ends,establishment_status,supervisor_no,supervisor_name,successor_no,successor_name
9180,PHARMACY,NEW YORK STATE DEPARTMENT OF CORRECTIONAL FACI...,NaN,WASHINGTON CORR. FACILITY C/O PHARMACY LOCK II...,19054,1986-04-10,2016-05-01,2016-04-22,NaN,NaN,NaN,NaN,NaN
9193,PHARMACY,NEW YORK STATE DEPARTMENT OF CORRECTIONAL SERV...,NaN,"GOUVERNEUR CORR. FACILITY C/O PHARMACY,P.O. BO...",20959,1991-05-15,2013-06-01,2010-05-27,NaN,NaN,NaN,NaN,NaN


In [103]:
pharmacies.query('establishment_status.isna()').shape

(5792, 13)

In [104]:
pharmacies.loc[(pharmacies.registration_ends > "2022-06-21"), 'active'] = True 
pharmacies.active = pharmacies.active.fillna(False)

In [105]:
pharmacies.active.value_counts()

False    9534
True     5367
Name: active, dtype: int64

In [106]:
# Active flag appears equal to null status
pharmacies.query('(active == True) & (~establishment_status.isna())')

,type,legal_name,trade_name,street_address,registration_number,date_first_registered,registration_begins,registration_ends,establishment_status,supervisor_no,supervisor_name,successor_no,successor_name,active
385,WHOLESLR NOT NYS,A & K DISTRIBUTOR PR LLC,NaN,"HANGAR #403 BROMON DR. AGUADILLA, PR 00603-0000",35803,2017-09-06,2020-09-01,2023-08-31,ACTIVE,NaN,NaN,NaN,NaN,True
390,PHARMACY NOT NYS,A & M PHARMACY LLC,NaN,"6564 WINTER GARDEN RD ORLANDO, FL 32835-0000",38454,2020-09-04,2020-09-04,2023-08-31,ACTIVE,NaN,NaN,NaN,NaN,True
417,PHARMACY NOT NYS,A PLUS PHARMACY & MEDICAL SUPPLY LLC,NaN,"1303 SE 17TH STREET STE D1 FORT LAUDERDALE, FL...",39050,2021-08-06,2021-08-06,2024-07-31,ACTIVE,NaN,NaN,NaN,NaN,True


In [109]:
pharmacies.type.value_counts()

PHARMACY               14880
WHOLESALER                 8
WHOLESLR NOT NYS           5
PHARMACY NOT NYS           4
MANUFACTURER               3
WHOLESALER/REPACKER        1
Name: type, dtype: int64

In [110]:
# Dropping wholesalers, manufacturers, out of state
pharmacies = pharmacies.query('type=="PHARMACY"')

In [111]:
pharmacies = pharmacies.reset_index().drop(columns='index')

In [74]:
pharmacies.shape

(14880, 14)

## Geocoding addresses

In [ ]:
api_key = getpass('Enter your HERE Location Services REST API key: ')

In [256]:
post_request = \
("https://batch.geocoder.ls.hereapi.com/6.2/jobs?"
f"apiKey={api_key}"
"&indelim=%7C"
"&outdelim=%7C"
"&action=run"
"&outcols=displayLatitude,displayLongitude,locationLabel,"
         "houseNumber,street,district,city,postalCode,county,state,country"
"&outputcombined=false")

In [257]:
post_body = "recId|searchText|country\n"

In [258]:
for index, address in enumerate(list(pharmacies.street_address)):
    if len(re.findall('\d.+', address)[0][:-5]) < 6:
        reformatted = str(index).zfill(4) + '|' + address[:-5] + '|USA\n'
    else:
        reformatted = str(index).zfill(4) + '|' + re.findall('\d.+', address)[0][:-5] + '|USA\n'
    post_body += reformatted

In [260]:
response = requests.post(post_request, post_body)

In [261]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ns2:SearchBatch xmlns:ns2="http://www.navteq.com/lbsp/Search-Batch/1"><Response><MetaInfo><RequestId>cJh8gj7IsKXLjKsCv8N3f6bQw9zT7Axx</RequestId></MetaInfo><Status>accepted</Status><TotalCount>0</TotalCount><ValidCount>0</ValidCount><InvalidCount>0</InvalidCount><ProcessedCount>0</ProcessedCount><PendingCount>0</PendingCount><SuccessCount>0</SuccessCount><ErrorCount>0</ErrorCount></Response></ns2:SearchBatch>'

In [262]:
request_id = re.findall('(?<=RequestId>)[A-Za-z0-9]+', response.text)[0]

In [263]:
status_request = f"https://batch.geocoder.ls.hereapi.com/6.2/jobs/{request_id}?action=status&apiKey={api_key}"

In [285]:
status = requests.get(status_request)

In [297]:
print(status.text)
print(re.findall('(?<=ErrorCount>)\d+', status.text)[0] + ' errors')

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ns2:SearchBatch xmlns:ns2="http://www.navteq.com/lbsp/Search-Batch/1"><Response><MetaInfo><RequestId>cJh8gj7IsKXLjKsCv8N3f6bQw9zT7Axx</RequestId></MetaInfo><Status>completed</Status><JobStarted>2022-06-23T23:14:28.000Z</JobStarted><JobFinished>2022-06-23T23:15:14.000Z</JobFinished><TotalCount>14880</TotalCount><ValidCount>14880</ValidCount><InvalidCount>1</InvalidCount><ProcessedCount>14880</ProcessedCount><PendingCount>0</PendingCount><SuccessCount>14870</SuccessCount><ErrorCount>10</ErrorCount></Response></ns2:SearchBatch>
10 errors


In [287]:
results_request = f'https://batch.geocoder.ls.hereapi.com/6.2/jobs/{request_id}/result?apiKey={api_key}'

In [292]:
result = requests.get(results_request)

In [301]:
result

<Response [200]>

In [302]:
wget.download(results_request)

100% [........................................................] 511998 / 511998

'results (3).zip'

In [303]:
geocoded = pd.read_csv('results (3).zip', sep='|')

In [304]:
geocoded.describe()

,recId,SeqNumber,seqLength,displayLatitude,displayLongitude,postalCode
count,15198.000000,15198.000000,15198.000000,15198.000000,15198.000000,15198.000000
mean,7435.320963,1.031715,1.063429,41.375352,-74.527569,11712.850836
std,4295.105460,0.260682,0.413694,0.994317,1.519491,1383.851967
min,0.000000,1.000000,1.000000,40.132934,-86.018925,10001.000000
25%,3716.250000,1.000000,1.000000,40.713200,-74.074928,10591.000000
50%,7432.500000,1.000000,1.000000,40.815290,-73.933970,11367.000000
75%,11138.750000,1.000000,1.000000,42.116810,-73.826329,12401.000000
max,14879.000000,10.000000,10.000000,44.992190,-71.937897,46034.000000


### Cleaning geocode outputs to only have one lat/long pair per pharmacy 

Use the zip code in the street address in the original data to match with the best geocoded option. 

In [311]:
# Outputs that had more than one lat/long possibility
geocoded.query('seqLength > 1').postalCode.value_counts()

11364    18
11355    17
11368    13
12065    12
11213    10
         ..
13662     1
13642     1
12581     1
12534     1
10128     1
Name: postalCode, Length: 183, dtype: int64

In [ ]:
pharmacies = pharmacies.rename_axis('recId').reset_index()

In [458]:
pharmacies_geo = pharmacies.merge(geocoded, on='recId',  how='outer')

In [366]:
zipcompare = pharmacies_geo.copy()

In [383]:
zipcompare['postalCode'] = zipcompare.street_address.str[-10:-5].astype(float)

In [462]:
conditions = [pharmacies_geo.postalCode == zipcompare.postalCode,
              pharmacies_geo.postalCode != zipcompare.postalCode]

choices = [True, False]

pharmacies_geo['zip_code_match'] = np.select(conditions, choices)

In [405]:
pharmacies_geo.query('(zip_code_match == 1) & (SeqNumber > 1)').recId.nunique()

180

In [419]:
pharmacies_geo.query('(zip_code_match == 1) & (SeqNumber == 1)').recId.nunique()

14408

In [420]:
pharmacies_geo.query('(zip_code_match == 0) & (SeqNumber == 1) & (seqLength == 1)').recId.nunique()

438

In [432]:
all_matches_wrong = list(\
pharmacies_geo.query('seqLength > 1').groupby(['recId', 'seqLength', 'zip_code_match'])\
.zip_code_match.count().reset_index(name='num')\
.query('(zip_code_match == 0) & (seqLength == num)').recId)

In [463]:
pharmacies_geo.drop(pharmacies_geo[(pharmacies_geo.zip_code_match == 0) & \
                                   (pharmacies_geo.seqLength > 1) & \
                                   (pharmacies_geo.recId.isin(all_matches_wrong)==False)]\
                                    .index, inplace=True)

In [465]:
pharmacies_geo[pharmacies_geo.recId.isin(all_matches_wrong)].recId.nunique()

22

In [466]:
pharmacies_geo.drop_duplicates(subset='recId', inplace=True)

In [ ]:
pharmacies_geo.to_csv('pharmacies_geocoded.csv')